In [2]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf LJSpeech-1.1.tar.bz2

--2025-05-24 18:31:41--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 169.150.247.33, 2400:52e0:1e00::1077:1
Connecting to data.keithito.com (data.keithito.com)|169.150.247.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2,6G) [text/plain]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2,56G  3,10MB/s    in 15m 5s  

2025-05-24 18:46:47 (2,90 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [ ]:
# HIFIGAN
import os

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import HifiganConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.gan import GAN

output_path = 'fine_tune_hifihan_ljspeech'

config = HifiganConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=False,
    test_delay_epochs=1,
    epochs=10,
    seq_len=4096,
    pad_short=2000,
    use_noise_augment=False,
    eval_split_size=5,
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    lr_gen=1e-4,
    lr_disc=1e-4,
    data_path="LJSpeech-1.1/wavs/",
    output_path=output_path,
)

# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
eval_samples, train_samples = load_wav_data(config.data_path, config.eval_split_size)

# init model
model = GAN(config, ap)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples[:4_000], eval_samples=eval_samples
)
trainer.fit()


In [1]:
import os

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import WavegradConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.wavegrad import Wavegrad

output_path = 'results/ft_wavegrad_ljspeech'

config = WavegradConfig(
    batch_size=64,                # Aggressive; safe for 24GB VRAM
    eval_batch_size=32,
    num_loader_workers=8,         # Use multiple CPU threads for I/O
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=5,
    epochs=10,                  # Full training cycle
    seq_len=16384,                # Very long sequences = better audio quality
    pad_short=3000,
    use_noise_augment=True,
    eval_split_size=100,          # Larger eval set for robust monitoring
    print_step=20,
    print_eval=True,
    mixed_precision=True,         # Essential for best performance on 4090
    data_path="LJSpeech-1.1/wavs/",
    output_path=output_path,
)

# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
eval_samples, train_samples = load_wav_data(config.data_path, config.eval_split_size)

# init model
model = Wavegrad(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=results/ft_wavegrad_ljspeech/run-May-24-2025_06+58PM-0000000
/mnt/Storage1/grozavu/conversion/.venv/lib/python3.11/site-packages/trainer/trainer.py:552: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()

 > Model h

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import WavernnConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.wavernn import Wavernn

output_path = 'results/ft_wavernn_ljspeech'
config = WavernnConfig(
    batch_size=128,
    eval_batch_size=32,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=10,
    seq_len=1280,
    pad_short=2000,
    use_noise_augment=True,
    eval_split_size=10,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    lr=1e-4,
    grad_clip=4,
    data_path='LJSpeech-1.1/wavs/',
    output_path=output_path,
)

# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
eval_samples, train_samples = load_wav_data(config.data_path, config.eval_split_size)

# init model
model = Wavernn(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
trainer.fit()


/tmp/ipykernel_17468/2655252565.py:39: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  model = Wavernn(config)
fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preempha

/mnt/Storage1/grozavu/conversion/.venv/lib/python3.11/site-packages/trainer/trainer.py:552: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()

 > Model has 4233673 parameters

 > EPOCH: 0/10
 --> results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000

 > TRAINING (2025-05-24 20:06:48) 

   --> TIME: 2025-05-24 20:06:58 -- STEP: 0/103 -- GLOBAL_STEP: 0
     | > loss: 11.831850051879883  (11.831850051879883)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(2.1972, device='cuda:0')  (tensor(2.1972, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 5.3197  (5.31970739364624)
     | > loader_time: 5.1933  (5.19328498840332)


   --> TIME: 2025-05-24 20:07:11 -- STEP: 25/103 -- GLOBAL_STEP: 25
     | > loss: 10.36344051361084  (10.679218139648437)
     | > amp_scaler: 2048.0  (23756.8)
     | > grad_norm: tensor(70.6353, device='cuda

 [!] Instance is too short! : LJSpeech-1.1/wavs/LJ017-0228.wav
156000/157300 -- batch_size: 13 -- gen_rate: 40.5 kHz -- x_realtime: 1.8  


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00010829501681857639 (+0)
     | > avg_loss: 9.038765483432346 (+0)

 > BEST MODEL : results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000/best_model_103.pth

 > EPOCH: 1/10
 --> results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000

 > TRAINING (2025-05-24 20:08:01) 

   --> TIME: 2025-05-24 20:08:21 -- STEP: 22/103 -- GLOBAL_STEP: 125
     | > loss: 9.188130378723145  (9.443207784132523)
     | > amp_scaler: 512.0  (558.5454545454546)
     | > grad_norm: tensor(105.0811, device='cuda:0')  (tensor(175.0742, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 0.4562  (0.49201423471624195)
     | > loader_time: 0.1126  (0.1704457781531594)


   --> TIME: 2025-05-24 20:08:37 -- STEP: 47/103 -- GLOBAL_STEP: 150
     | > loss: 9.734430313110352  (9.326689091134575)
     | > amp_scaler: 512.0  (533.787234042553)
     | > grad_norm: tensor(225.5271, device='cuda:0')  (tensor(166.5952, device='cuda:0'))
     | > cu

...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00010959307352701823 (+1.7192628648546014e-05)
     | > avg_loss: 9.177064365810818 (+0.41258637110392193)


 > EPOCH: 8/10
 --> results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000

 > TRAINING (2025-05-24 20:16:24) 

   --> TIME: 2025-05-24 20:16:30 -- STEP: 1/103 -- GLOBAL_STEP: 825
     | > loss: 10.409693717956543  (10.409693717956543)
     | > amp_scaler: 256.0  (256.0)
     | > grad_norm: tensor(391.4507, device='cuda:0')  (tensor(391.4507, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 0.5769  (0.5768787860870361)
     | > loader_time: 0.0909  (0.09088611602783203)


   --> TIME: 2025-05-24 20:16:47 -- STEP: 26/103 -- GLOBAL_STEP: 850
     | > loss: 8.758657455444336  (9.232333696805515)
     | > amp_scaler: 256.0  (256.0)
     | > grad_norm: tensor(252.8665, device='cuda:0')  (tensor(151.7598, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 0.5093  (0.4967046609291664)
     | > lo

 [!] Instance is too short! : LJSpeech-1.1/wavs/LJ017-0228.wav
156000/157300 -- batch_size: 13 -- gen_rate: 43.2 kHz -- x_realtime: 2.0  


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00010450681050618489 (-5.086263020833342e-06)
     | > avg_loss: 9.025825182596842 (-0.1512391832139759)


 > EPOCH: 9/10
 --> results/ft_wavernn_ljspeech/run-May-24-2025_08+06PM-0000000

 > TRAINING (2025-05-24 20:17:35) 

   --> TIME: 2025-05-24 20:17:57 -- STEP: 23/103 -- GLOBAL_STEP: 950
     | > loss: 9.140965461730957  (9.261809680772865)
     | > amp_scaler: 256.0  (256.0)
     | > grad_norm: tensor(64.2867, device='cuda:0')  (tensor(233.1356, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 0.4581  (0.4955996741419253)
     | > loader_time: 0.0966  (0.1504283676976743)


   --> TIME: 2025-05-24 20:18:13 -- STEP: 48/103 -- GLOBAL_STEP: 975
     | > loss: 8.860940933227539  (9.050653616587322)
     | > amp_scaler: 256.0  (256.0)
     | > grad_norm: tensor(150.8796, device='cuda:0')  (tensor(201.2847, device='cuda:0'))
     | > current_lr: 0.0001 
     | > step_time: 0.4065  (0.46619660158952075)
     | > loader

 [!] Instance is too short! : LJSpeech-1.1/wavs/LJ017-0228.wav
156000/157300 -- batch_size: 13 -- gen_rate: 40.9 kHz -- x_realtime: 1.9  


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00010246700710720486 (-2.0398033989800317e-06)
     | > avg_loss: 9.001180330912272 (-0.024644851684570312)



In [8]:
# CUSTOM DATASET

import os

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import WavegradConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.wavegrad import Wavegrad

output_path = 'results/ft_wavegrad_custom_ljspeech'

config = WavegradConfig(
    batch_size=64,                # Aggressive; safe for 24GB VRAM
    eval_batch_size=32,
    num_loader_workers=8,         # Use multiple CPU threads for I/O
    num_eval_loader_workers=4,
    run_eval=False,
    test_delay_epochs=100,  # No eval during training
    epochs=50,                  # Full training cycle
    seq_len=16384,                # Very long sequences = better audio quality
    pad_short=3000,
    use_noise_augment=True,
    eval_split_size=0,          # Larger eval set for robust monitoring
    print_step=20,
    print_eval=True,
    mixed_precision=True,         # Essential for best performance on 4090
    data_path="Custom_LJSpeech/wavs",
    output_path=output_path,
)

# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
eval_samples, train_samples = load_wav_data(config.data_path, config.eval_split_size)

# init model
model = Wavegrad(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=results/ft_wavegrad_custom_ljspeech/run-May-25-2025_02+27PM-0000000
/mnt/Storage1/grozavu/conversion/.venv/lib/python3.11/site-packages/trainer/trainer.py:552: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()

 > 